In [1]:
import torch

class Layer(torch.nn.Module):
    def __init__(self, inputDim, outputDim, activation):
        
        super(Layer, self).__init__()
        self.inputDim = inputDim
        self.outputDim = outputDim
        self.linear = torch.nn.Linear(inputDim, outputDim)
        activation_functions = [torch.nn.Identity, torch.nn.ReLU, torch.nn.Tanh, torch.nn.Sigmoid]
        activation_inputs = ["Identity", "ReLU", "Tanh", "Sigmoid"]
        self.activation = activation_functions[activation_inputs.index(activation)]()

        
    def forward(self, x):
        inp = self.linear(x)
        output = self.activation(inp)
        
        return output


In [2]:
class Neural_Network(torch.nn.Module):
    def __init__(self, layers):
        super(Neural_Network, self).__init__()
        self.layers = layers
        lst = []
        for layer in layers:
            lst += list(layer.parameters())
        self.params = torch.nn.ParameterList(lst)
        
    def forward(self, x):
        temp_input = x
        for layer in self.layers:
            temp_input = layer.forward(temp_input)
        return temp_input
    def predict(self, data):
        data_tensor = torch.FloatTensor(data)
        return [torch.argmax(self.forward(item)).item() for item in data_tensor]
    def score(self, x_test, y_test):
        x_test_tensor = torch.FloatTensor(pd.DataFrame(x_test).values)
        test_results = self.forward(x_test_tensor)
        results = [torch.argmax(res).item() for res in test_results]
        
        total = 0
        for i in range(len(results)):
            if results[i] == y_test[i]:
                total += 1

        return total/len(y_test)

    

In [3]:
def nn_construct(layer_dims, activation):
    
    return Neural_Network([Layer(layer_dims[i-1], layer_dims[i], activation) for i in range(1, len(layer_dims))])


In [68]:
import pandas as pd
import numpy as np
def nn_trainer(model, X_train, y_train):
    # Trains a model
    # Input: model: the neural network to be trained (Neural_Network)
    #        X_train: training for features
    #        y_train: training data for target
    criterion = torch.nn.CrossEntropyLoss()

    learning_rate = 1e-4
    optimizer = torch.optim.Adam(model.params, lr=learning_rate)

    for t in range(5000):
        # Forward pass: compute predicted y by passing x to the model.
        x_train_tensor = torch.FloatTensor(X_train)
        
        y_pred = model.forward(x_train_tensor)
        
        # Compute and print loss.
        loss = criterion(y_pred, torch.tensor(pd.DataFrame(y_train).values, dtype = torch.long).reshape(-1))

        # Before the backward pass, use the optimizer object to zero all of the
        # gradients for the variables it will update (which are the learnable
        # weights of the model). This is because by default, gradients are
        # accumulated in buffers( i.e, not overwritten) whenever .backward()
        # is called. Checkout docs of torch.autograd.backward for more details.
        optimizer.zero_grad()

        # Backward pass: compute gradient of the loss with respect to model
        # parameters
        loss.backward()

        # Calling the step function on an Optimizer makes an update to its
        # parameters
        optimizer.step()
    
    return model.parameters(), model.forward(x_train_tensor)

In [69]:
from sklearn.model_selection import train_test_split

In [70]:
# Reads data
features = pd.read_csv("data/features.csv")
target = pd.read_csv("data/target.csv")

In [71]:
# Bins target values
from sklearn.preprocessing import KBinsDiscretizer

discretizer = KBinsDiscretizer(n_bins=4, encode = "onehot-dense", strategy = "quantile")

target_discretized = discretizer.fit_transform(target["G3"].values.reshape(-1, 1))
target_new = []
for row in target_discretized:
    target_new.append(list(row).index(1))

In [72]:
# Splits data into training, testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target_new, random_state = 1001)
X_train = X_train.values

In [73]:
model = nn_construct([96, 48, 4], "Sigmoid")

In [74]:
params, activations = nn_trainer(model, X_train, y_train)

In [75]:
nn_test_score = model.score(X_test,y_test)

0.44061302681992337

In [76]:
nn_train_score = model.score(X_train, y_train)

0.5951468710089399